# Use Sedona and orsm to calculate route distance and duration

In previous tutorials, we have calculated the bird view distance between two points. If we need to real route distance and duration, the default Sedona function can't do that.
 
In this tutorial, we will use sedona and [orsm-backend](https://github.com/Project-OSRM/osrm-backend) to find best route to destination, then calculate distance and duration.

> a doc on how to deploy orsm https://github.com/pengfei99/OSRM-deployement

The dataset is always the French commune data set released by INSEE.

Step1: calculate the centroid of each french commune
Step2: convert the centroid (geometry point) to a GPS coordinates(double),OSRM-backend exposes a rest api
Step3: Build a start point, end point matrix
Step4: Create a spark udf
Step5: Use the udf to calculate the distance and duration 


In [1]:
from sedona.spark import *
from sedona.sql import st_functions as stf
from pathlib import Path
import requests
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

Skipping SedonaKepler import, verify if keplergl is installed
Skipping SedonaPyDeck import, verify if pydeck is installed


In [2]:
# build a sedona session offline
jar_folder = Path(r"/home/pengfei/git/PySparkCommonFunc/jars")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.5.1)
config = SedonaContext.builder() \
    .master("local[*]") \
    .config("spark.driver.memory","6G") \
    .config('spark.jars', jar_path). \
    getOrCreate()
# config = SedonaContext.builder(). \
#     config('spark.jars.packages',
#            'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.4.1,'
#            'org.datasyslab:geotools-wrapper:1.4.0-28.2'). \
#     getOrCreate()

# create a sedona context
sedona = SedonaContext.create(config)

24/04/24 09:54:33 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
24/04/24 09:54:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/24 09:54:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Step 0: Intro of the OSRM API

Get the distance and duration by using osrm api. Here we only use the simple one
 
A testing query

```shell
# below is the general form
# route/v1 is the main corps
# driving indicates which mode of the route we are asking. followed by the gps coordinates of the starting point and 
# end point
# steps can be true or false, if true, the reponse will contain the route itinerary
curl "http://<host>:<port>/route/v1/driving/<start_longitude>,<start_latitude>;<end_longitude>,<end_latitude>?steps=true"

# an example
# start_gps = lat:48.819552,long:2.309167
# end_gps = lat:48.758568,long:2.467290
curl "https://maps-api.casd.local/route/v1/driving/2.309167,48.819552;2.467290,48.758568?steps=false"
```

You should receive the below response
The route clause is the route itinerary which osrm returned. In it, you can find:
- "distance": 18677.9 : It means the route distance between the two points is 18677.9 meters
- "duration": 1115.1: It means the itinerary duration is 1115.1 seconds or 18 minutes

> The duration is calculated by using the max authorized speed without considering traffic jam and other complications. So in reality, the itinerary should take more time.

```json
{
  "code": "Ok",
  "routes": [
    {
      "geometry": "g`~hHc_bM^eDiRk\\zVinBrIqSdBy_AcIge@vHi\\bAyc@o_@}_B_W_i@{Ik`@tWua@dIaS`U{rAnBemAs@}oAbEan@lGeK`RgGfh@p@nUfFbi@b_@dY~Iz_@fBxj@oDrl@ee@vNgSbN__@lQk{@lLsRpL_I",
      "legs": [
        {
          "steps": [],
          "summary": "",
          "weight": 1115.1,
          "duration": 1115.1,
          "distance": 18677.9
        }
      ],
      "weight_name": "routability",
      "weight": 1115.1,
      "duration": 1115.1,
      "distance": 18677.9
    }
  ],
  "waypoints": [
    {
      "hint": "zrIqgdCyKoEIAAAARwAAANsAAAAAAAAAD1xjQMnb60GQULZCAAAAAAgAAABHAAAA2wAAAAAAAABtIgAAEDwjAMfs6AIvPCMAYO3oAgkAHwbkNR0k",
      "distance": 17.166158355,
      "name": "",
      "location": [2.309136, 48.819399]
    },
    {
      "hint": "V5UEgP___38OAAAAMQAAAJQBAACcAgAAzx6vQSYxWULcY0pElQ6ZRA4AAAAxAAAAlAEAAJwCAABtIgAAja8lAD8B6ALapSUAKP_nAhIAHwrkNR0k",
      "distance": 191.955244684,
      "name": "",
      "location": [2.469773, 48.759103]
    }
  ]
}
```


In [4]:
! curl -k "https://maps-api.casd.local/route/v1/driving/2.309167,48.819552;2.467290,48.758568?steps=false"

{"code":"Ok","routes":[{"geometry":"g`~hHc_bM^eDiRk\\zVinBrIqSdBy_AcIge@vHi\\bAyc@o_@}_B_W_i@{Ik`@tWua@dIaS`U{rAnBemAs@}oAbEan@lGeK`RgGfh@p@nUfFbi@b_@dY~Iz_@fBxj@oDrl@ee@vNgSbN__@lQk{@lLsRpL_I","legs":[{"steps":[],"summary":"","weight":1115.1,"duration":1115.1,"distance":18677.9}],"weight_name":"routability","weight":1115.1,"duration":1115.1,"distance":18677.9}],"waypoints":[{"hint":"zrIqgdCyKoEIAAAARwAAANsAAAAAAAAAD1xjQMnb60GQULZCAAAAAAgAAABHAAAA2wAAAAAAAABtIgAAEDwjAMfs6AIvPCMAYO3oAgkAHwbkNR0k","distance":17.166158355,"name":"","location":[2.309136,48.819399]},{"hint":"V5UEgP___38OAAAAMQAAAJQBAACcAgAAzx6vQSYxWULcY0pElQ6ZRA4AAAAxAAAAlAEAAJwCAABtIgAAja8lAD8B6ALapSUAKP_nAhIAHwrkNR0k","distance":191.955244684,"name":"","location":[2.469773,48.759103]}]}

## Step 1: calculate the centroid of each french commune

In [5]:
fr_zone_file_path= "/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet"

In [6]:
fr_zone_df = sedona.read.format("geoparquet").load(fr_zone_file_path)
fr_zone_df.cache()
fr_zone_df.show()

+--------------------+--------------------+--------------------+-----------------+-----+
|            geometry|           wikipedia|             surf_ha|              nom|insee|
+--------------------+--------------------+--------------------+-----------------+-----+
|POLYGON ((9.32016...|fr:Pie-d'Orezza  ...|     573.00000000...|     Pie-d'Orezza|2B222|
|POLYGON ((9.20010...|fr:Lano          ...|     824.00000000...|             Lano|2B137|
|POLYGON ((9.27757...|fr:Cambia        ...|     833.00000000...|           Cambia|2B051|
|POLYGON ((9.25119...|fr:Érone         ...|     393.00000000...|            Érone|2B106|
|POLYGON ((9.28339...|fr:Oletta        ...|    2674.00000000...|           Oletta|2B185|
|POLYGON ((9.30951...|fr:Canari (Haute-...|    1678.00000000...|           Canari|2B058|
|POLYGON ((9.30101...|fr:Olmeta-di-Tuda...|    1753.00000000...|   Olmeta-di-Tuda|2B188|
|POLYGON ((9.32662...|fr:Campana       ...|     236.00000000...|          Campana|2B052|
|POLYGON ((9.33944...

In [7]:
centroid_df = fr_zone_df.withColumn("centroid",stf.ST_Centroid(col("geometry"))).select("nom","insee","centroid").withColumnRenamed("centroid","geometry")

In [8]:
centroid_df.show()

+-----------------+-----+--------------------+
|              nom|insee|            geometry|
+-----------------+-----+--------------------+
|     Pie-d'Orezza|2B222|POINT (9.33815086...|
|             Lano|2B137|POINT (9.23535777...|
|           Cambia|2B051|POINT (9.30210765...|
|            Érone|2B106|POINT (9.26661425...|
|           Oletta|2B185|POINT (9.33384508...|
|           Canari|2B058|POINT (9.34524454...|
|   Olmeta-di-Tuda|2B188|POINT (9.36394979...|
|          Campana|2B052|POINT (9.34042768...|
|Carcheto-Brustico|2B063|POINT (9.36026336...|
|         Ampriani|2B015|POINT (9.35701808...|
|         Pianello|2B213|POINT (9.35641690...|
|            Zuani|2B364|POINT (9.34092227...|
|     Pietraserena|2B226|POINT (9.35471346...|
|     Piedipartino|2B221|POINT (9.34491216...|
|         Montbolo|66113|POINT (2.63221051...|
|       Targasonne|66202|POINT (1.98851907...|
|         L'Albère|66001|POINT (2.89587079...|
|       Mont-Louis|66117|POINT (2.11967214...|
|          Es

## Step2: Convert the centroid

Convert the centroid (geometry point) to a GPS coordinates(double),OSRM-backend exposes a rest api

In [11]:
converted_centroid_df = centroid_df.withColumn("longitude",stf.ST_X(col("geometry"))).withColumn("latitude",stf.ST_Y(col("geometry"))).drop("geometry")

In [16]:
converted_centroid_df.cache()
converted_centroid_df.show()

+-----------------+-----+------------------+------------------+
|              nom|insee|         longitude|          latitude|
+-----------------+-----+------------------+------------------+
|     Pie-d'Orezza|2B222| 9.338150861836196|42.374292014354154|
|             Lano|2B137| 9.235357777014519| 42.37887024991088|
|           Cambia|2B051| 9.302107656444328| 42.36875223806091|
|            Érone|2B106|  9.26661425039706|42.375563316535825|
|           Oletta|2B185|  9.33384508224219|42.641774511917404|
|           Canari|2B058| 9.345244547654016|42.843017113153394|
|   Olmeta-di-Tuda|2B188| 9.363949798662757| 42.61232393952698|
|          Campana|2B052| 9.340427687694566| 42.38826970859529|
|Carcheto-Brustico|2B063| 9.360263365997817| 42.35520610104405|
|         Ampriani|2B015| 9.357018084967732|  42.2540399256354|
|         Pianello|2B213| 9.356416901011539| 42.29772067884147|
|            Zuani|2B364| 9.340922275473961| 42.26546661866408|
|     Pietraserena|2B226| 9.354713461428

In [13]:
converted_centroid_df.printSchema()

root
 |-- nom: string (nullable = true)
 |-- insee: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)


## Step 3: Build the matrix  

In [17]:
# build a commune code list, which will be used as starting point of the matrix
insee_code_list = ["75056","92049"]
commune_df = converted_centroid_df.filter(col("insee").isin(insee_code_list))
commune_df.show()

+---------+-----+------------------+-----------------+
|      nom|insee|         longitude|         latitude|
+---------+-----+------------------+-----------------+
|    Paris|75056|2.3428764301940275|48.85662219553845|
|Montrouge|92049|2.3171758940549156|48.81520615999795|
+---------+-----+------------------+-----------------+


In [19]:
commune_matrix_df = (commune_df.alias("add1")
                  .join(converted_centroid_df.alias("add2"),col("add1.insee")!=col("add2.insee"),"inner")
                  .select(col("add1.longitude").alias("source_long"),col("add1.latitude").alias("source_lat"),col("add1.insee").alias("source_insee"),col("add1.nom").alias("source_nom"),col("add2.longitude").alias("dest_long"),col("add2.latitude").alias("dest_lat"),col("add2.insee").alias("dest_insee"),col("add2.nom").alias("dest_nom")))
commune_matrix_df.show()

+------------------+-----------------+------------+----------+------------------+------------------+----------+-----------------+
|       source_long|       source_lat|source_insee|source_nom|         dest_long|          dest_lat|dest_insee|         dest_nom|
+------------------+-----------------+------------+----------+------------------+------------------+----------+-----------------+
|2.3428764301940275|48.85662219553845|       75056|     Paris| 9.338150861836196|42.374292014354154|     2B222|     Pie-d'Orezza|
|2.3428764301940275|48.85662219553845|       75056|     Paris| 9.235357777014519| 42.37887024991088|     2B137|             Lano|
|2.3428764301940275|48.85662219553845|       75056|     Paris| 9.302107656444328| 42.36875223806091|     2B051|           Cambia|
|2.3428764301940275|48.85662219553845|       75056|     Paris|  9.26661425039706|42.375563316535825|     2B106|            Érone|
|2.3428764301940275|48.85662219553845|       75056|     Paris|  9.33384508224219|42.641774

> The commune_matrix_df contains two starting point "75056(paris)","92049(montrouge)", and endpoints are all other french coummnes

In [20]:
commune_matrix_df.count()

69908

# Step4: Create a spark udf

There are two ways to declare spark udf, here I used the annotation appraoch

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf


@udf(returnType=StringType()) 
def get_distance_duration(lat_start:str,long_start:str,lat_end:str,long_end:str):
    return calculate_distance_duration_str(lat_start,long_start,lat_end,long_end)

In [21]:
def get_route(lat_start:str, long_start:str, lat_end:str, long_end:str, show_steps:str="false")->dict:
    """
    This function takes a starting point and end point gps coordinates, then call the osrm rest api.
    It returns the api json response if the response status is 200, otherwise return None.
    :param lat_start: 
    :type lat_start: 
    :param long_start: 
    :type long_start: 
    :param lat_end: 
    :type lat_end: 
    :param long_end: 
    :type long_end: 
    :param show_steps: 
    :type show_steps: 
    :return: 
    :rtype: 
    """
    host="maps-api.casd.local"
    start_point = f"{long_start},{lat_start}"
    end_point= f"{long_end},{lat_end}"
    # Define the URL
    url = f"https://{host}/route/v1/driving/{start_point};{end_point}?steps={show_steps}"
    
    # Make the GET request
    response = requests.get(url,verify=False)
    json_response = None
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response content
        json_response = response.json()
    else:
        print("Error:", response.status_code)
    return json_response

In [22]:
# an example of get_route
start_long = "2.309167"
start_lat = "48.819552"
end_long = "2.467290"
end_lat = "48.758568"
route_json = get_route(start_lat,start_long,end_lat,end_long)

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [24]:
print(type(route_json))
print(route_json)

<class 'dict'>
{'code': 'Ok', 'routes': [{'geometry': 'g`~hHc_bM^eDiRk\\zVinBrIqSdBy_AcIge@vHi\\bAyc@o_@}_B_W_i@{Ik`@tWua@dIaS`U{rAnBemAs@}oAbEan@lGeK`RgGfh@p@nUfFbi@b_@dY~Iz_@fBxj@oDrl@ee@vNgSbN__@lQk{@lLsRpL_I', 'legs': [{'steps': [], 'summary': '', 'weight': 1115.1, 'duration': 1115.1, 'distance': 18677.9}], 'weight_name': 'routability', 'weight': 1115.1, 'duration': 1115.1, 'distance': 18677.9}], 'waypoints': [{'hint': 'zrIqgdCyKoEIAAAARwAAANsAAAAAAAAAD1xjQMnb60GQULZCAAAAAAgAAABHAAAA2wAAAAAAAABtIgAAEDwjAMfs6AIvPCMAYO3oAgkAHwbkNR0k', 'distance': 17.166158355, 'name': '', 'location': [2.309136, 48.819399]}, {'hint': 'V5UEgP___38OAAAAMQAAAJQBAACcAgAAzx6vQSYxWULcY0pElQ6ZRA4AAAAxAAAAlAEAAJwCAABtIgAAja8lAD8B6ALapSUAKP_nAhIAHwrkNR0k', 'distance': 191.955244684, 'name': '', 'location': [2.469773, 48.759103]}]}


In [25]:
def parse_route_json(input_route:dict)->(float,float):
    """
    This function parse the orsm json response, and return distance(meter), duration(minute)
    :param input_route: 
    :type input_route: 
    :return: tuple of distance and duration
    :rtype: (float,float)
    """
    route = input_route['routes'][0]
    if route:
        # the raw distance is in meter
        distance = route["distance"]
        # the raw duration is in second
        # the returned duration is in minutes
        duration = round((route["duration"]/60), 2)
    else:
        distance = None
        duration = None
    return distance, duration

In [26]:
# an example of parse_route_json
dis1,dur1= parse_route_json(route_json)

In [29]:
print(f"distance has type: {type(dis1)}, value: {dis1}")
print(f"duration has type: {type(dur1)}, value: {dur1}")


distance has type: <class 'float'>, value: 18677.9
duration has type: <class 'float'>, value: 18.58


In [18]:
def calculate_distance_duration(lat_start:str,long_start:str,lat_end:str,long_end:str)->(float,float):
    """
    This function takes a starting point and end point gps coordinates, then call the osrm rest api. It
    parses the response and returns the distance(meter) and duration(minutes)
    :param lat_start: 
    :type lat_start: 
    :param long_start: 
    :type long_start: 
    :param lat_end: 
    :type lat_end: 
    :param long_end: 
    :type long_end: 
    :return: 
    :rtype: 
    """
    route = get_route(lat_start,long_start,lat_end,long_end)
    distance, duration= parse_route_json(route)
    return distance, duration  

In [19]:
def calculate_distance_duration_str(lat_start:str,long_start:str,lat_end:str,long_end:str)->str:
    route = get_route(lat_start,long_start,lat_end,long_end)
    distance, duration= parse_route_json(route)
    return f"{distance};{duration}"


In [21]:
 def calculate_distance_matrix(insee_code_list:list, centroid_df:DataFrame, output_file_path:str):
    for insee_code in insee_code_list:
        commune_df = centroid_df.filter(col("insee")==insee_code)
        commune_matrix=commune_df.alias("add1").join(centroid_df.alias("add2"),col("add1.insee")!=col("add2.insee"),"inner").select(col("add1.longitude").alias("source_long"),col("add1.latitude").alias("source_lat"),col("add1.insee").alias("source_insee"),col("add1.nom").alias("source_nom"),col("add2.longitude").alias("dest_long"),col("add2.latitude").alias("dest_lat"),col("add2.insee").alias("dest_insee"),col("add2.nom").alias("dest_nom"))
        commune_matrix.show()

In [22]:
distance1, duration1=calculate_distance_duration("48.819552","2.309167","48.728568","2.447290")

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [23]:
result = calculate_distance_duration_str("48.819552","2.309167","48.728568","2.447290")
print(result)

19654.1,23.04


/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [24]:
print("Distance:", distance1, "meters")
print("Duration:", duration1, "minutes")

Distance: 19654.1 meters
Duration: 23.04 minutes


bastil = 48.85329591862303, 2.3694278896689758
home_gps = 48.728568,2.447290

In [61]:
distance2, duration2=calculate_distance_duration("48.85329591862303","2.3694278896689758","48.728568","2.447290")

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [45]:
print("Distance:", distance2, "meters")
print("Duration:", duration2, "minutes")

Distance: 16856.2 meters
Duration: 25.14333333333333 minutes


## Build commune matrix



In [38]:
commune_insee_code_list = [row.insee for row in centroid_df.select("insee").distinct().collect()]
print(commune_insee_code_list[0])

24/04/23 16:53:44 WARN MemoryStore: Not enough space to cache rdd_11_2 in memory! (computed 68.2 MiB so far)
24/04/23 16:53:44 WARN BlockManager: Persisting block rdd_11_2 to disk instead.
24/04/23 16:53:45 WARN MemoryStore: Not enough space to cache rdd_11_2 in memory! (computed 68.2 MiB so far)


2B226


In [41]:
commune_df = test_df.filter(col("insee")=="92049")
commune_matrix=commune_df.alias("add1").join(test_df.alias("add2"),col("add1.insee")!=col("add2.insee"),"inner").select(col("add1.longitude").alias("source_long"),col("add1.latitude").alias("source_lat"),col("add1.insee").alias("source_insee"),col("add1.nom").alias("source_nom"),col("add2.longitude").alias("dest_long"),col("add2.latitude").alias("dest_lat"),col("add2.insee").alias("dest_insee"),col("add2.nom").alias("dest_nom"))


+------------------+-----------------+------------+----------+------------------+------------------+----------+-----------------+
|       source_long|       source_lat|source_insee|source_nom|         dest_long|          dest_lat|dest_insee|         dest_nom|
+------------------+-----------------+------------+----------+------------------+------------------+----------+-----------------+
|2.3171758940549156|48.81520615999795|       92049| Montrouge| 9.338150861836196|42.374292014354154|     2B222|     Pie-d'Orezza|
|2.3171758940549156|48.81520615999795|       92049| Montrouge| 9.235357777014519| 42.37887024991088|     2B137|             Lano|
|2.3171758940549156|48.81520615999795|       92049| Montrouge| 9.302107656444328| 42.36875223806091|     2B051|           Cambia|
|2.3171758940549156|48.81520615999795|       92049| Montrouge|  9.26661425039706|42.375563316535825|     2B106|            Érone|
|2.3171758940549156|48.81520615999795|       92049| Montrouge|  9.33384508224219|42.641774

In [44]:
distance_matrix = commune_matrix.withColumn("distance",get_distance_duration(col("source_lat"),col("source_long"),col("dest_lat"),col("dest_long")))
distance_matrix.show()

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

+------------------+-----------------+------------+----------+------------------+------------------+----------+-----------------+----------------+
|       source_long|       source_lat|source_insee|source_nom|         dest_long|          dest_lat|dest_insee|         dest_nom|        distance|
+------------------+-----------------+------------+----------+------------------+------------------+----------+-----------------+----------------+
|2.3171758940549156|48.81520615999795|       92049| Montrouge| 9.338150861836196|42.374292014354154|     2B222|     Pie-d'Orezza|1183176.9,907.83|
|2.3171758940549156|48.81520615999795|       92049| Montrouge| 9.235357777014519| 42.37887024991088|     2B137|             Lano|1170398.6,892.96|
|2.3171758940549156|48.81520615999795|       92049| Montrouge| 9.302107656444328| 42.36875223806091|     2B051|           Cambia|1175794.5,900.05|
|2.3171758940549156|48.81520615999795|       92049| Montrouge|  9.26661425039706|42.375563316535825|     2B106|       

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

In [12]:
%%time

lat_start=48.853295
long_start=2.369427
lat_end= 48.728568 
long_end=2.447290
i=0
while i<100:
    decal=i*0.0001
    lat_start= lat_start+decal
    long_start= long_start+decal
    lat_end= lat_end+decal
    long_end= long_end+decal
    distance3, duration3=calculate_distance_duration(f"{str(lat_start)}",f"{long_start}",f"{lat_end}",f"{long_end}")
    i=i+1
    print("Distance:", distance3, "meters")
    print("Duration:", duration3, "minutes")


/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

Distance: 16856.3 meters
Duration: 25.14 minutes
Distance: 16849.3 meters
Duration: 25.13 minutes
Distance: 17201.2 meters
Duration: 26.27 minutes
Distance: 17547.7 meters
Duration: 27.32 minutes
Distance: 17450.9 meters
Duration: 27.09 minutes
Distance: 17449 meters
Duration: 27.0 minutes
Distance: 17350.3 meters
Duration: 26.73 minutes
Distance: 17462.3 meters
Duration: 26.85 minutes
Distance: 17369.9 meters
Duration: 26.33 minutes
Distance: 17685.2 meters
Duration: 26.93 minutes
Distance: 17912 meters
Duration: 26.16 minutes
Distance: 17987.4 meters
Duration: 26.6 minutes
Distance: 18241.3 meters
Duration: 27.59 minutes
Distance: 18600.4 meters
Duration: 28.47 minutes
Distance: 19464.9 meters
Duration: 30.79 minutes


/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

Distance: 18535.6 meters
Duration: 29.55 minutes
Distance: 19218.9 meters
Duration: 28.88 minutes
Distance: 18827.5 meters
Duration: 30.39 minutes
Distance: 19654.1 meters
Duration: 29.74 minutes
Distance: 19844.2 meters
Duration: 30.84 minutes
Distance: 20206.9 meters
Duration: 27.33 minutes
Distance: 22492.1 meters
Duration: 26.04 minutes
Distance: 22521.9 meters
Duration: 25.82 minutes
Distance: 21377.8 meters
Duration: 23.23 minutes
Distance: 23840.1 meters
Duration: 26.18 minutes
Distance: 21527.4 meters
Duration: 25.62 minutes
Distance: 22148.7 meters
Duration: 23.93 minutes
Distance: 21490.8 meters
Duration: 24.53 minutes
Distance: 20891 meters
Duration: 26.72 minutes
Distance: 24446.9 meters
Duration: 28.87 minutes
Distance: 22167.6 meters
Duration: 29.62 minutes
Distance: 24376.5 meters
Duration: 28.54 minutes
Distance: 25947.6 meters
Duration: 33.37 minutes
Distance: 18780.6 meters
Duration: 27.51 minutes
Distance: 18565.1 meters
Duration: 26.19 minutes
Distance: 17723.6 mete

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

...

/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

Distance: 26791.7 meters
Duration: 27.65 minutes
Distance: 28791.5 meters
Duration: 31.73 minutes
Distance: 27192.3 meters
Duration: 29.78 minutes
Distance: 25000.7 meters
Duration: 26.7 minutes
Distance: 22048.9 meters
Duration: 22.89 minutes
Distance: 22238.7 meters
Duration: 27.14 minutes
Distance: 24247.8 meters
Duration: 24.05 minutes
Distance: 21843.8 meters
Duration: 24.43 minutes
Distance: 20365.2 meters
Duration: 21.88 minutes
Distance: 21902.1 meters
Duration: 25.02 minutes
Distance: 24148.6 meters
Duration: 30.74 minutes
Distance: 25284 meters
Duration: 33.06 minutes
Distance: 19563.3 meters
Duration: 24.23 minutes
Distance: 18718.8 meters
Duration: 27.5 minutes
Distance: 18764.6 meters
Duration: 30.36 minutes


/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Add

Distance: 19956.2 meters
Duration: 28.95 minutes
Distance: 21746.5 meters
Duration: 28.03 minutes
Distance: 22173.2 meters
Duration: 27.49 minutes
Distance: 22481.5 meters
Duration: 29.47 minutes
Distance: 24577.5 meters
Duration: 25.68 minutes
Distance: 21531.7 meters
Duration: 29.0 minutes
Distance: 23294.8 meters
Duration: 28.17 minutes
Distance: 21237.4 meters
Duration: 28.09 minutes
Distance: 19752.4 meters
Duration: 26.77 minutes
Distance: 25022.4 meters
Duration: 31.7 minutes
Distance: 22843.1 meters
Duration: 27.51 minutes
Distance: 23033.9 meters
Duration: 29.45 minutes
CPU times: user 436 ms, sys: 70.3 ms, total: 506 ms
Wall time: 1.28 s


/home/pengfei/.cache/pypoetry/virtualenvs/geoparquetandsedona-uqJLJxB7-py3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps-api.casd.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
